In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [11]:
df = pd.read_csv('horseRacing.csv')
df

,race_id,horse_id,1-101,1-102,1-103,1-104,1-105,1-106,1-107,1-108,...,3-132,4-105,4-109,4-110,4-114,4-115,4-116,4-117,calc_position,price
0,495886,565129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.71,...,-1.16,0.0,1.87,1.87,-2.070,-1.96,-1.950,-0.30,8.0,17.0
1,495886,607093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.05,...,0.77,0.0,0.00,0.00,0.000,0.00,0.000,0.07,5.0,26.0
2,495886,637337,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.97,...,-1.16,0.0,0.00,0.00,0.480,0.44,0.390,-0.86,7.0,17.0
3,495886,659819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.21,...,0.77,0.0,0.00,0.00,0.320,1.09,0.780,-0.86,1.0,4.0
4,495886,661530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08,...,0.77,0.0,0.00,0.00,0.000,0.00,0.390,1.17,4.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98385,691981,1770390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,11.0
98386,691981,1770391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,81.0
98387,719014,901847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.71,-0.71,0.740,0.71,0.770,0.71,1.0,3.1
98388,719014,904955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.71,0.71,-0.670,-0.71,-0.640,-0.71,2.0,7.0


In [12]:
df = df.dropna()
label = df['calc_position']
df = df.drop(['race_id','horse_id','calc_position'], axis = 1)

In [ ]:
class MLP:
    def __init__(self, layers, activation='sigmoid'):
        self.layers = layers
        self.activation = activation
        self.weights = []
        self.biases = []
        for i in range(1, len(layers)):
            self.weights.append(np.random.randn(layers[i-1], layers[i]))
            self.biases.append(np.random.randn(layers[i]))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def softmax(self, x):
        exp_x = np.exp(x)
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)
    
    def forward(self, x):
        a = x
        for i in range(len(self.layers)-1):
            z = np.dot(a, self.weights[i]) + self.biases[i]
            if self.activation == 'sigmoid':
                a = self.sigmoid(z)
            elif self.activation == 'relu':
                a = self.relu(z)
            else:
                raise ValueError('Invalid activation function.')
        return a
    
    def cross_entropy_loss(self, y_pred, y_true):
        m = y_true.shape[0]
        logp = -np.log(y_pred[np.arange(m), y_true.argmax(axis=1)])
        loss = np.sum(logp) / m
        return loss
    
    def backward(self, x, y_true, y_pred, learning_rate):
        m = y_true.shape[0]
        dz = y_pred - y_true
        for i in range(len(self.layers)-2, -1, -1):
            dw = np.dot(self.layers[i], dz) / m
            db = np.mean(dz, axis=0)
            if self.activation == 'sigmoid':
                dz = np.dot(dz, self.weights[i].T) * self.sigmoid(self.layers[i], derivative=True)
            elif self.activation == 'relu':
                dz = np.dot(dz, self.weights[i].T) * self.relu(self.layers[i], derivative=True)
            self.weights[i] -= learning_rate * dw
            self.biases[i] -= learning_rate * db
    
    def train(self, X, y, learning_rate=0.1, epochs=1000):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = self.cross_entropy_loss(y_pred, y)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")
            self.backward(X, y, y_pred, learning_rate)
    
    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.2, random_state=42)

mlp = MLP(layers=[2, 4, 3], activation='sigmoid')
mlp.train(X_train, y_train, learning_rate=0.1, epochs=1000)